# 🎯 Structured Output Demo

This notebook demonstrates how to use **structured output** with Fence's Links, Chains, and Agents. Structured output ensures you get properly formatted, typed data instead of raw strings.

## What You'll Learn

- How to define Pydantic models for structured output
- Using structured output with Links
- Chaining structured outputs together  

## Setup

First, let's set up our environment and imports.

In [1]:
from pathlib import Path
current_dir = Path('.').resolve().parents[0]
import sys
sys.path.append(str(current_dir))

%load_ext autoreload
%autoreload 2

In [2]:
from pydantic import BaseModel, Field
from typing import List
from fence.models.bedrock import NovaPro
from fence.links import Link
from fence.chains import Chain
from fence.templates.string import StringTemplate

## Define Pydantic Models for Structured Output

First, let's define the data structures we want our models to return. These Pydantic models ensure type safety and validation.

In [3]:
class ProductReview(BaseModel):
    """Structured product review analysis."""

    rating: int = Field(..., ge=1, le=5, description="Rating from 1-5 stars")
    sentiment: str = Field(
        ..., description="Overall sentiment: positive, negative, or neutral"
    )
    key_points: List[str] = Field(..., description="Key points mentioned in the review")
    recommendation: str = Field(..., description="Recommendation based on the review")


class CompetitorAnalysis(BaseModel):
    """Structured competitor analysis."""

    competitor_name: str = Field(..., description="Name of the competitor")
    strengths: List[str] = Field(..., description="Competitor's key strengths")
    weaknesses: List[str] = Field(..., description="Competitor's key weaknesses")
    market_position: str = Field(
        ..., description="Market position: leader, challenger, follower, or niche"
    )
    threat_level: int = Field(..., ge=1, le=5, description="Threat level from 1-5")


class MarketInsight(BaseModel):
    """Structured market insight."""

    product_name: str = Field(..., description="Product being analyzed")
    overall_rating: float = Field(..., ge=1.0, le=5.0, description="Average rating")
    total_reviews: int = Field(..., description="Number of reviews analyzed")
    sentiment_summary: str = Field(..., description="Overall sentiment summary")
    top_competitors: List[str] = Field(..., description="Top 3 competitors identified")
    market_recommendation: str = Field(..., description="Strategic recommendation")


print("✅ Pydantic models defined!")
print(f"📋 ProductReview fields: {list(ProductReview.__fields__.keys())}")
print(f"🏢 CompetitorAnalysis fields: {list(CompetitorAnalysis.__fields__.keys())}")
print(f"📊 MarketInsight fields: {list(MarketInsight.__fields__.keys())}")

✅ Pydantic models defined!
📋 ProductReview fields: ['rating', 'sentiment', 'key_points', 'recommendation']
🏢 CompetitorAnalysis fields: ['competitor_name', 'strengths', 'weaknesses', 'market_position', 'threat_level']
📊 MarketInsight fields: ['product_name', 'overall_rating', 'total_reviews', 'sentiment_summary', 'top_competitors', 'market_recommendation']


/var/folders/x0/8q2p5vcd6356_kq623qdnc7h0000gp/T/ipykernel_59024/974015492.py:36: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print(f"📋 ProductReview fields: {list(ProductReview.__fields__.keys())}")
/var/folders/x0/8q2p5vcd6356_kq623qdnc7h0000gp/T/ipykernel_59024/974015492.py:37: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print(f"🏢 CompetitorAnalysis fields: {list(CompetitorAnalysis.__fields__.keys())}")
/var/folders/x0/8q2p5vcd6356_kq623qdnc7h0000gp/T/ipykernel_59024/974015492.py:38: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed

## Demo 1: Structured Output with Links 🔗

Let's start with the basics - using structured output with individual Links.

In [4]:
# Create a model with structured output
model = NovaPro(
    source="structured_demo", region="us-east-1", output_structure=ProductReview
)

# Create a link for review analysis
review_analyzer = Link(
    model=model,
    template=StringTemplate(
        """
    Analyze this product review and extract structured information:
    
    Review: "{review_text}"
    
    Please analyze the sentiment, rating, key points, and provide a recommendation.
    """
    ),
    name="review_analyzer",
)

print("🔗 Review analyzer Link created!")
print(f"📝 Model: {model.model_name}")
print(f"🎯 Output Structure: {model.output_structure.__name__}")

🔗 Review analyzer Link created!
📝 Model: Nova Pro
🎯 Output Structure: ProductReview


In [5]:
# Test with sample reviews
sample_reviews = [
    "This laptop is amazing! Super fast, great battery life, and the screen is gorgeous. Definitely worth the money. 5 stars!",
    "The phone is okay but the battery dies too quickly. Camera is decent though. Would give it 3 stars.",
    "Terrible product. Broke after one week. Customer service was unhelpful. Don't waste your money. 1 star.",
    "I switched from iPhone to this Samsung Galaxy and I'm impressed! The camera is better than my old iPhone 13, battery lasts all day, and the price was much more reasonable. The only downside is some apps work better on iOS, but overall very happy. 9/10!",
]

for i, review in enumerate(sample_reviews, 1):
    print(f"\n📝 Review {i}:")
    print(f"Text: {review}")

    result = review_analyzer.run(review_text=review)
    result = result[
        "state"
    ]  # 🎯 Dictionary with 'state' key containing the structured data

    print(f"\n✨ Structured Analysis:")
    print(f"   Rating: {result['rating']}/5 ⭐")
    print(f"   Sentiment: {result['sentiment']} 😊")
    print(f"   Key Points: {', '.join(result['key_points'])} 📌")
    print(f"   Recommendation: {result['recommendation']} 💡")


📝 Review 1:
Text: This laptop is amazing! Super fast, great battery life, and the screen is gorgeous. Definitely worth the money. 5 stars!

✨ Structured Analysis:
   Rating: 5/5 ⭐
   Sentiment: positive 😊
   Key Points: super fast, great battery life, gorgeous screen, worth the money 📌
   Recommendation: highly recommended 💡

📝 Review 2:
Text: The phone is okay but the battery dies too quickly. Camera is decent though. Would give it 3 stars.

✨ Structured Analysis:
   Rating: 3/5 ⭐
   Sentiment: neutral 😊
   Key Points: battery dies too quickly, camera is decent 📌
   Recommendation: Would give it 3 stars 💡

📝 Review 3:
Text: Terrible product. Broke after one week. Customer service was unhelpful. Don't waste your money. 1 star.

✨ Structured Analysis:
   Rating: 1/5 ⭐
   Sentiment: negative 😊
   Key Points: Terrible product, Broke after one week, Customer service was unhelpful, Don't waste your money 📌
   Recommendation: Do not purchase 💡

📝 Review 4:
Text: I switched from iPhone to th

## Demo 2: Structured Output with Chains ⛓️

Now let's see how to chain multiple structured outputs together for complex workflows.

In [6]:
# Step 1: Analyze individual reviews
review_model = NovaPro(source="chain_demo", region="us-east-1")

review_link = Link(
    model=review_model,
    template=StringTemplate(
        """
    Analyze this product review:
    "{review_text}"
    """
    ),
    name="review_analyzer",
    output_key="review_analysis",
    output_structure=ProductReview,  # 🎯 Structured output defined in Link!
)

print("🔗 Step 1: Review analyzer created")

🔗 Step 1: Review analyzer created


In [7]:
# Step 2: Analyze competitors mentioned
competitor_model = NovaPro(source="chain_demo", region="us-east-1")

competitor_link = Link(
    model=competitor_model,
    template=StringTemplate(
        """
    Based on this review analysis, identify and analyze the main competitor mentioned:
    
    Review Analysis: {review_analysis}
    
    If no specific competitor is mentioned, analyze the general market leader in this category.
    """
    ),
    name="competitor_analyzer",
    output_key="competitor_analysis",
    output_structure=CompetitorAnalysis,  # 🎯 Structured output defined in Link!
)

print("🏢 Step 2: Competitor analyzer created")

🏢 Step 2: Competitor analyzer created


In [8]:
# Step 3: Generate market insights
insight_model = NovaPro(source="chain_demo", region="us-east-1")

insight_link = Link(
    model=insight_model,
    template=StringTemplate(
        """
    Generate market insights based on this analysis:
    
    Review Analysis: {review_analysis}
    Competitor Analysis: {competitor_analysis}
    
    Provide strategic recommendations for the product.
    """
    ),
    name="insight_generator",
    output_structure=MarketInsight,  # 🎯 Structured output defined in Link!
)

print("📊 Step 3: Market insight generator created")

📊 Step 3: Market insight generator created


In [9]:
# Create the chain
analysis_chain = Chain(links=[review_link, competitor_link, insight_link])

print("⛓️ Analysis chain created with 3 structured output steps!")
print(f"🔗 Links: {[link.name for link in analysis_chain.links]}")

⛓️ Analysis chain created with 3 structured output steps!
🔗 Links: ['review_analyzer', 'competitor_analyzer', 'insight_generator']


In [10]:
# Run the chain with a sample review
sample_review = """
I switched from iPhone to this Samsung Galaxy and I'm impressed! 
The camera is better than my old iPhone 13, battery lasts all day, 
and the price was much more reasonable. The only downside is some 
apps work better on iOS, but overall very happy. 4.5 stars!
"""

print(f"📱 Analyzing Review: {sample_review.strip()[:100]}...")
print("\n🔄 Running chain...")

result = analysis_chain.run(input_dict={"review_text": sample_review})
final_insight = result["state"]

print(f"\n🎯 Market Insight Generated:")
print(f"   📱 Product: {final_insight['product_name']}")
print(f"   ⭐ Overall Rating: {final_insight['overall_rating']}/5.0")
print(f"   😊 Sentiment: {final_insight['sentiment_summary']}")
print(f"   🏢 Top Competitors: {', '.join(final_insight['top_competitors'])}")
print(f"   💡 Recommendation: {final_insight['market_recommendation']}")
print(f"   📊 Total Reviews Analyzed: {final_insight['total_reviews']}")

📱 Analyzing Review: I switched from iPhone to this Samsung Galaxy and I'm impressed! 
The camera is better than my old i...

🔄 Running chain...

🎯 Market Insight Generated:
   📱 Product: state
   ⭐ Overall Rating: 4.0/5.0
   😊 Sentiment: positive
   🏢 Top Competitors: iPhone
   💡 Recommendation: 
   📊 Total Reviews Analyzed: 1


## Demo 3: Advanced Parsing - FAQ Generation from Text 🔧

This demo shows how `enable_advanced_parsing` helps handle:
- **Case-insensitive field matching** (PascalCase, camelCase, snake_case)
- **Multiple tool outputs** that need to be merged into lists
- **Nested models** with complex structures

We'll extract question-answer pairs from a long passage of text.

In [15]:
from fence.models.bedrock import NovaLite


# Define models for FAQ generation
class QuestionAnswerContext(BaseModel):
    """A single question-answer pair with context."""

    question: str = Field(..., description="The question to be answered")
    answer: str = Field(..., description="The answer to the question")
    context: str = Field(
        ..., description="A quote from the text that contains the answer"
    )


class FAQOutput(BaseModel):
    """FAQ generation output with multiple Q&A pairs."""

    topic: str = Field(..., description="Main topic of the text")
    qa_pairs: List[QuestionAnswerContext] = Field(
        ..., description="List of question-answer-context objects"
    )


print("✅ FAQ models defined!")

✅ FAQ models defined!


In [16]:
# Sample text about a topic (Düsseldorf)
sample_text = """
Düsseldorf is the capital city of North Rhine-Westphalia, the most populous state 
of Germany. It is the second-largest city in the state after Cologne and the 
sixth-largest city in Germany, with a 2024 population of 618,685. Most of Düsseldorf 
lies on the right bank of the Rhine, and the city has grown together with Neuss, 
Ratingen, Meerbusch, Erkrath and Monheim.

Düsseldorf is the central city of Rhine-Ruhr, the second biggest metropolitan region 
by GDP in the European Union, that stretches from Bonn via Cologne and Düsseldorf 
to the Ruhr.

Düsseldorf began as a small settlement at the mouth of the Düssel River, gaining 
city rights in 1288 after the Battle of Worringen. In the Late Middle Ages and 
Renaissance, it grew into a regional political and cultural centre under the Dukes 
of Berg and later as the capital of the United Duchies of Jülich-Cleves-Berg.

The 17th and 18th centuries saw flourishing arts and architecture, with Düsseldorf 
becoming known for its court culture and early art academy. During the Napoleonic 
era, it was briefly part of the Grand Duchy of Berg before coming under Prussian 
control in 1815, where it industrialised rapidly in the 19th century.

During the 20th century, Düsseldorf developed into one of Germany's most important 
centres for administration, business, and culture. In 1946, it became the capital 
of the newly created state of North Rhine-Westphalia, strengthening its political role.
"""

print(f"📄 Sample text loaded ({len(sample_text)} characters)")
print(f"📝 Preview: {sample_text[:150]}...")

📄 Sample text loaded (1456 characters)
📝 Preview: 
Düsseldorf is the capital city of North Rhine-Westphalia, the most populous state 
of Germany. It is the second-largest city in the state after Colog...


In [17]:
# Create model with advanced parsing enabled (default)
faq_model = NovaLite(
    source="faq_demo",
    region="us-east-1",
    output_structure=FAQOutput,
    enable_advanced_parsing=True,  # 🎯 This enables case-insensitive matching and merging!
)

faq_link = Link(
    model=faq_model,
    template=StringTemplate(
        """
    Generate FAQ questions and answers from the following text.
    
    Text:
    ```
    {text}
    ```
    
    Create {num_questions} question-answer pairs. For each pair:
    - Question: A clear, specific question about the text
    - Answer: A concise answer (1-2 sentences max)
    - Context: The exact quote from the text that supports the answer
    
    Make sure all questions can be answered directly from the text.
    """
    ),
    name="faq_generator",
)

print("🔗 FAQ generator Link created with advanced parsing enabled!")
print(f"📝 Model: {faq_model.model_name}")
print(f"🎯 Output Structure: {faq_model.output_structure.__name__}")
print(f"⚙️ Advanced Parsing: {faq_model.enable_advanced_parsing}")

🔗 FAQ generator Link created with advanced parsing enabled!
📝 Model: Nova Lite
🎯 Output Structure: FAQOutput
⚙️ Advanced Parsing: True


In [19]:
# Generate FAQs from the text
print("🔄 Generating FAQ from text...\n")

result = faq_link.run(text=sample_text, num_questions=5)
faq_output = result["state"]

print("✨ FAQ Generated!\n")
print(f"📚 Topic: {faq_output['topic']}")
print("=" * 80)

for i, qa in enumerate(faq_output["qa_pairs"], 1):
    print(f"\n❓ Question {i}: {qa['question']}")
    print(f"✅ Answer: {qa['answer']}")
    print(f"📝 Context: \"{qa['context']}\"")
    print("-" * 80)

🔄 Generating FAQ from text...

✨ FAQ Generated!

📚 Topic: Düsseldorf Information

❓ Question 1: What is the population of Düsseldorf in 2024?
✅ Answer: 618,685
📝 Context: "with a 2024 population of 618,685."
--------------------------------------------------------------------------------

❓ Question 2: In which state of Germany is Düsseldorf located?
✅ Answer: North Rhine-Westphalia
📝 Context: "Düsseldorf is the capital city of North Rhine-Westphalia, the most populous state of Germany."
--------------------------------------------------------------------------------

❓ Question 3: What is the historical significance of Düsseldorf gaining city rights?
✅ Answer: In 1288 after the Battle of Worringen.
📝 Context: "gaining city rights in 1288 after the Battle of Worringen."
--------------------------------------------------------------------------------

❓ Question 4: What was Düsseldorf known for during the 17th and 18th centuries?
✅ Answer: Flourishing arts and architecture, court cultur

### Why Advanced Parsing Matters 🎯

**Without `enable_advanced_parsing=True`, you might encounter:**

1. **Case Sensitivity Errors**: Bedrock might return `{"Topic": "...", "QaPairs": [...]}` (PascalCase) but Pydantic expects `{"topic": "...", "qa_pairs": [...]}` (snake_case)

2. **Multiple Tool Outputs**: Sometimes Bedrock returns multiple tool use calls instead of a single call with a list. For example, instead of returning:
   ```python
   {"qa_pairs": [{"question": "Q1", ...}, {"question": "Q2", ...}]}
   ```
   It might return 2 separate tool calls:
   ```python
   {"qa_pairs": [{"question": "Q1", ...}]}
   {"qa_pairs": [{"question": "Q2", ...}]}
   ```
   Advanced parsing merges these intelligently into a single list.

3. **Nested Model Issues**: Without proper alias handling, nested models like `QuestionAnswerContext` inside `FAQOutput` might fail validation if field names don't match exactly.

**With `enable_advanced_parsing=True` (default):**
- ✅ Automatically adds validation aliases for all case variations (PascalCase, camelCase, snake_case, etc.)
- ✅ Merges multiple tool outputs into list fields intelligently
- ✅ Handles nested models recursively
- ✅ More robust and forgiving of API response variations
- ✅ Reduces validation errors and improves reliability

In [20]:
# Compare: Model WITHOUT advanced parsing (not recommended)
print("⚠️ Creating model WITHOUT advanced parsing (for comparison)...\n")

strict_model = NovaLite(
    source="strict_demo",
    region="us-east-1",
    output_structure=FAQOutput,
    enable_advanced_parsing=False,  # ❌ Disabled - may fail with case mismatches!
)

print("⚙️ Strict model created (advanced parsing disabled)")
print("⚠️ This model may fail if Bedrock returns fields in PascalCase or camelCase!")
print("⚠️ It will also only use the first tool output if multiple are returned.")
print(
    "⚠️ Try running the same FAQ generation with this model - it might fail or return incomplete results!"
)

⚠️ Creating model WITHOUT advanced parsing (for comparison)...

⚙️ Strict model created (advanced parsing disabled)
⚠️ This model may fail if Bedrock returns fields in PascalCase or camelCase!
⚠️ It will also only use the first tool output if multiple are returned.
⚠️ Try running the same FAQ generation with this model - it might fail or return incomplete results!
